In [5]:
import os
import pandas as pd
from blockbaxAnalyser import blockbaxAnalyser
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# --------------- SETTINGS -----------------------------------------
verbose = True # Set to True to get intermediate prints of results
plots = False
sb_numbers = [148098, 148097, 148091, 148088, 148076, 148096]
temp_steps = [-20, -10, 0, 10, 20, 30, 40, 50]
temp_steps_short = [-10, 0, 10, 20]
y_steps = [-10, -5, -2, 0, 2, 5, 10]
bins_y = [-11, -7, -3.5, -1, 1, 3.5, 7, 10]
bins_temp = [-25, -15, -5, 5, 15, 25, 35, 45, 55]
tl_min = -2 # min x value of trendline
tl_max = 2 # max x value of trendline

plot_from_date = "2022-04-28 18:30:00"
plot_till_date = "2022-05-05 01:00:00"

In [7]:
# --------------- INITIALISATION --------------------------------------
analyser = blockbaxAnalyser(sb_numbers = sb_numbers, 
plot_from_date=plot_from_date, 
plot_till_date=plot_till_date)

analyser.bins_y = bins_y
analyser.bins_temp = bins_temp
analyser.y_steps = y_steps
analyser.temp_steps = temp_steps

analyser.dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
offset_date_start="2022-05-02 03:40:00",
offset_date_end="2022-05-02 05:35:00")

if verbose:
    i=0
    print(analyser.dfs)
    print(sb_numbers[i])
    print(analyser.dfs[i].to_string())

[                               Datetime  Temperature   Y value  Reference  \
Datetime                                                                    
2022-04-28 18:31:00 2022-04-28 18:31:00   -16.799999  9.713613   9.380461   
2022-04-28 18:36:01 2022-04-28 18:36:01   -17.400000  9.731923   9.387061   
2022-04-28 18:41:00 2022-04-28 18:41:00   -17.799999  9.719716   9.370361   
2022-04-28 18:46:02 2022-04-28 18:46:02   -18.200001  9.731923   9.382961   
2022-04-28 18:51:00 2022-04-28 18:51:00   -18.500000  9.725820   9.368061   
...                                 ...          ...       ...        ...   
2022-05-05 00:15:51 2022-05-05 00:15:51    51.099998  8.810292   9.121761   
2022-05-05 00:20:51 2022-05-05 00:20:51    51.099998  8.816396   9.115361   
2022-05-05 00:25:52 2022-05-05 00:25:52    51.099998  8.810292   9.096661   
2022-05-05 00:30:51 2022-05-05 00:30:51    51.099998  8.804189   9.127461   
2022-05-05 00:35:51 2022-05-05 00:35:51    51.099998  8.804189   9.124261  

Hieronder is de data visueel uitgezet. Elk plotje is van één specifieke smartbrick (zie legenda voor serienummer) bij één specifieke hoek. Per smartbrick zijn er dus 7 plots. Dit omdat de sterke correlatie die eerder ontdekt was ook bij een test was waarbij we één hoek aanhielden en verschillende temperaturen testten.

In [8]:
# -------------------- PLOT STANDARD TRANSFORMED DATA (T, Y) ------------------------------
for i in range(len(sb_numbers)):
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf'], label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Yref stdtf'], label="Reference")
            if analyser.slope != None:
                x = np.linspace(-2,2,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature (STDTF)')
            plt.ylabel('Angle Y (STDTF)')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None   

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -1.051867, intercept: 0.323932
R-squared: 0.853725
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -1.677272, intercept: 0.320663
R-squared: 0.931426
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -1.691321, intercept: 0.240459
R-squared: 0.909782
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -1.044263, intercept: 0.060376
R-squared: 0.463262
--------------------------------------------------------------------

Op het oog vallen er een paar dingen op (*Vragen*):
1. Geen erg sterke correlatie tussen temperatuur en Y voor alle SmartBricks
2. Bij de ingesteld hoek van 10 graden loopt de genormaliseerde temperatuur sterk uiteen.
3. Er zijn verschillen met de referentie, maar de referentie vertoont ook correlatie met de temperatuur. Aangezien de referentie in principe temperatuursgecompenseerd is en we deze als absolute waarheid nemen, kunnen we concluderen dat dit attribueerbaar is aan de beweging van de testopstelling door temperatuursinvloed (dus niet door de invloed van temperatuur op de sensor).

*Antwoorden*
1. Verdeel in groepen gebaseerd op R2 en slope
    - Gem. positieve slope: 148076, 148096, 148088 
    - Gem. negatieve slope: 148098, 148091, 148097
    - Hoge R2 (gem. >0.8): 148098,148076
    - Meh R2 (rond 0.5): 148091, 148097, 148088, 148096 
2. Analyse hieronder wijst uit dat dit komt door hysterese bij -10 graden inclinatie en -20 graden temperatuur
3. klopt
Opmerking: Een verschil met de vorige analyse is dat in de vorige analyse met genormaliseerde data alléén de Y genormaliseerd was, niet de temperatuur. (dit is hieronder geprobeerd)

In [9]:
# ------------------------- NON-STDTF TEMPERATURE -------------------------------
for i in range(len(sb_numbers)):
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y_notf_temp(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temperature'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf'], label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temperature'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Yref stdtf'], label='reference')

            if analyser.slope != None:
                x = np.linspace(-20,50,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature')
            plt.ylabel('Angle Y')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -0.048555, intercept: 0.980123
R-squared: 0.984474
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -0.058663, intercept: 1.124488
R-squared: 0.991064
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -0.054655, intercept: 1.190514
R-squared: 0.984499
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -0.033154, intercept: 0.683424
R-squared: 0.455353
--------------------------------------------------------------------

Uit deze plots zien we een redelijk sterke correlatie tussen de temperatuur en standaard getransformeerde Y. 
Hieruit wordt ook duidelijk dat de grote spreiding die waargenomen was in het vorige block volledig attribueerbaar is aan de hysterese die plaatsvindt omdat de y stappen [10, 5, 2, 0, 2, -2, -5, -10, 0, 10] zijn, en dus dat de hoek 10 2x voorkomt. Wat opmerkelijk is is dat de hysterese bij de 0 hoek wel over de hele temperatuursrange aanwezig is, maar bij de 10 hoek alleen bij de temperatuur -20.
Ook is er bij de 10 graden hoek een groot verschil tussen het gedrag van de referentie en het gedrag van de SmartBricks, waar deze zich op andere temperaturen redelijk hetzelfde gedragen.
*Conclusies*
- Gebaseerd op de bovenstaande plots kunnen we geen groepen maken; Alle smartbricks hebben een relatief hoge R2 en een negatieve slope, waarbij de R2 ronduit slecht is bij hoek 10 (zoals hierboven beschreven attribueerbaar aan de hysterese bij temperatuur -20)
- Vergeleken met standaard getransformeerde temperatuur:

*Vragen*
- Gebruiken we standaard getransformeerde temperatuur of ongetransformeerde temperatuur?
- Moeten we met het model de error (= verschil tussen referentie en gemeten/getransformeerde hoek) of de hoek proberen te voorspellen?

*Antwoorden*
- Ongetransformeerde temperatuur geeft betere correlatie maar getransformeerde temperatuur geeft beter inzicht in het model, ik ga voor nu verder met getransformeerde temperatuur
- Het model moet de error voorspellen, zie hieronder voor analyse op basis van error


In [10]:
# -------------------- PLOT STANDARD TRANSFORMED DATA ERROR (T, Y) ------------------------------
plots=False
for i in range(len(sb_numbers)):
    analyser.dfs[i]["Y stdtf error"] = analyser.dfs[i]["Y stdtf"] - analyser.dfs[i]["Yref stdtf"]
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y_error(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], 
            analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf error'], 
            label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            if analyser.slope != None:
                x = np.linspace(-2,2,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature')
            plt.ylabel('Angle Y')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None 

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -0.125193, intercept: 0.175615
R-squared: 0.642831
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -0.474648, intercept: 0.188541
R-squared: 0.867962
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -0.406747, intercept: 0.255812
R-squared: 0.708680
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -0.542619, intercept: 0.091066
R-squared: 0.811842
--------------------------------------------------------------------

De conclusie hieruit is dat er wel een verdeling gemaakt kan worden uit het error-gedrag van de smartbricks, maar dat deze niet direct correspondeert met wat er uit de analyse van ruwe data komt. Dit is een probleem omdat we de smartbricks dan niet in het veld kunnen classificeren (aangezien voor het bepalen van de error zonder a priori kennis een referentie nodig is).

In [11]:
# ---------------------------------- MODEL PREDICTS ERROR (REF-MEAS) VALUE --------------------------------------------------------
# One model for positive correlation and one for negative correlation
sbs_positive = [148076, 148096, 148088,148098, 148091, 148097]
sbs_negative = [148098, 148091, 148097]
group_names = ["positively correlated", "negatively correlated"]
groups = [sbs_positive]
stdtf = True
analysers = []

print("MODEL PREDICTS ERROR (REF-MEAS) VALUE")
for n in range(len(groups)):
    print("Results for group: ", group_names[n], "-------------------------------------------------------------------------------------")
    analysers.append(blockbaxAnalyser(sb_numbers= groups[n],
    plot_from_date=plot_from_date, 
    plot_till_date=plot_till_date))
    analysers[n].y_steps = y_steps
    analysers[n].bins_y = bins_y
    analysers[n].temp_steps = temp_steps
    analysers[n].bins_temp = bins_temp

    analysers[n].dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
        offset_date_start="2022-05-02 03:40:00",
        offset_date_end="2022-05-02 05:35:00")
    
    analysers[n].trainerror(stdtf=stdtf)
    
    analysers[n].test(error=True)
    
    if plots:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        # FIRST: Plot surface from model
        x0_mesh = np.arange(-12, 12, 0.01) # Measured angle mesh
        x1_mesh = np.arange(-22, 55, 0.1) # Measured temperature mesh
        x0_mesh, x1_mesh = np.meshgrid(x0_mesh, x1_mesh)

        y_mesh = analysers[n].model.coef_[0]*1 + analysers[n].model.coef_[1]*x0_mesh + analysers[n].model.coef_[2]*x1_mesh + analysers[n].model.coef_[3]*x0_mesh*x0_mesh + analysers[n].model.coef_[4]*x0_mesh*x1_mesh + analysers[n].model.coef_[5]*x1_mesh*x1_mesh

        ax.plot_surface(x0_mesh, x1_mesh, y_mesh) # find out transparant surface map

        # SECOND: Plot scattered data
        for i in range(len(groups[n])):
            ax.scatter(analysers[n].dfs[i]['Y value'], analysers[n].dfs[i]['Temperature'], analysers[n].dfs[i]['Reference']-analysers[n].dfs[i]['Y value'], color=analysers[n].colours[i], label=analysers[n].sb_numbers[i])

        # Cleanup
        ax.set_xlabel("Measured angle")
        ax.set_ylabel("Measured temperature")
        ax.set_zlabel("Ref - meas angle")
        ax.legend()
        plt.show()

MODEL PREDICTS ERROR (REF-MEAS) VALUE
Results for group:  positively correlated -------------------------------------------------------------------------------------
Polynomial fit with regression to data of all Smartbricks, inputs and references standard transformed
Serial numbers:  [148076, 148096, 148088, 148098, 148091, 148097]
Feature names:  ['1' 'x0' 'x1' 'x0^2' 'x0 x1' 'x1^2']
x0 = Measured angle
x1 = Temperature
Model coefficients:  [-0.03135525 -0.29007847  0.10089354  0.03317218  0.12820945  0.01591077]
148076 ---------------------------------------
Max absolute uncompensated error 1.107963512975095
Average uncompensated error:  -0.14977959194833265
Min absolute uncompensated error -1.1477977929229723
Max absolute compensated error:  0.7054089000970691
Average compensated error:  -0.15295408586046638
Min absolute compensated error:  -1.1436705815326658
RMSE uncompensated:  0.27234583295765125
RMSE compensated:  0.2718392181035205
Absolute improvement:  0.0005066148541307269


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# FIRST: Plot surface from model
x0_mesh = np.arange(-12, 12, 0.01) # Measured angle mesh
x1_mesh = np.arange(-22, 55, 0.1) # Measured temperature mesh
x0_mesh, x1_mesh = np.meshgrid(x0_mesh, x1_mesh)

y_mesh = analysers[n].model.coef_[0]*1 + analysers[n].model.coef_[1]*x0_mesh + analysers[n].model.coef_[2]*x1_mesh + analysers[n].model.coef_[3]*x0_mesh*x0_mesh + analysers[n].model.coef_[4]*x0_mesh*x1_mesh + analysers[n].model.coef_[5]*x1_mesh*x1_mesh

ax.plot_surface(x0_mesh, x1_mesh, y_mesh) # find out transparant surface map

# SECOND: Plot scattered data
for i in range(len(self.sb_numbers)):
    ax.scatter(analysers[n].dfs[i]['Y value'], analysers[n].dfs[i]['Temperature'], analysers[n].dfs[i]['Reference']-analysers[n].dfs[i]['Y value'], color=analysers[n].colours[i], label=analysers[n].sb_numbers[i])

# Cleanup
ax.set_xlabel("Measured angle")
ax.set_ylabel("Measured temperature")
ax.set_zlabel("Ref - meas angle")
ax.legend()
plt.show()

In [8]:
# ---------------------------------- MODEL PREDICTS TRUE (REF) VALUE --------------------------------------------------------

print("MODEL PREDICTS TRUE (REF) VALUE")
for n in range(len(groups)):
    print("Results for group: ", group_names[n], "-------------------------------------------------------------------------------------")
    analysers.append(blockbaxAnalyser(sb_numbers= groups[n],
    plot_from_date=plot_from_date, 
    plot_till_date=plot_till_date))
    analysers[n].y_steps = y_steps
    analysers[n].bins_y = bins_y
    analysers[n].temp_steps = temp_steps
    analysers[n].bins_temp = bins_temp

    analysers[n].dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
        offset_date_start="2022-05-02 03:40:00",
        offset_date_end="2022-05-02 05:35:00")
    
    analysers[n].trainreference(stdtf=stdtf)
    
    analysers[n].test(reference=True)

Results for group:  positively correlated -------------------------------------------------------------------------------------
Polynomial fit with regression to data of all Smartbricks, inputs and references standard transformed
Serial numbers:  [148076, 148096, 148088]
Feature names:  ['1' 'x0' 'x1' 'x0^2' 'x0 x1' 'x1^2']
x0 = Measured angle
x1 = Temperature
Model coefficients:  [-0.04547977  0.71315476  0.07090546  0.04591693  0.18809194  0.00876967]
148076 ---------------------------------------
Max absolute uncompensated error 1.3742500747132471
Average uncompensated error:  -0.1302083193876786
Min absolute uncompensated error -1.0792306009012194
Max absolute compensated error:  2.246627234263503
Average compensated error:  -0.17382985426519051
Min absolute compensated error:  -1.19328449863254
RMSE uncompensated:  0.2500730776763395
RMSE compensated:  0.39267035708943765
Absolute improvement:  -0.14259727941309813
(Negative improvement means deterioration.)
 
148096 -------------

To do list
[x] Group data based on y setting and temp setting
[x] Normalise data based on stdtransform
[x] Plot normalised data and determine R2 and trendline
[x] Analyse and evaluate
[x] Group smartbricks based on R2 and trendline slope/intersect
[ ] Create model for well-correlated smartbricks